-  how to get hepmass data from publink link: http://archive.ics.uci.edu/ml/datasets/hepmass#
-  in this code you format the dataset and reduce the size down to 500K
-  link to the resulting dataset: 
        - 500K: https://drive.google.com/file/d/1EKjJcqCrLY2m92L6cEhfgEAIswrnEyqJ/view?usp=sharing
        - 5K: https://drive.google.com/file/d/1oID5w-TH2r3IDKBhlut_6WCn8DTVNsaZ/view?usp=sharing

This code will be updated with the standardization procedure

In [52]:
import pandas as pd
import numpy as np
# public link:   
pth_train = "../../../../../Downloads/1000_train.csv"
all_train = pd.read_csv(pth_train)

In [61]:
row = np.zeros(len(all_train.columns), dtype = int)
row[0] = 100 # random for now for the ranbox code to read, to be changed
row[1] = 100
smalldataset =  all_train.iloc[:500_000]; smalldataset.columns = row;
smallerdataset =  all_train.iloc[:5000]; smallerdataset.columns = row;

In [62]:
smalldataset.to_csv("hepmass_train_500K.csv", header = True, index = False, sep = " ", float_format = '{:.6E}'.format)
smallerdataset.to_csv("hepmass_train_5K.csv", header = True, index = False, sep = " ", float_format = '{:.6E}'.format)

In [63]:
pd.read_csv("hepmass_train_500K.csv")

,100 100 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
0,2.178961E-01 9.330718E-01 -5.961744E-01 1.1484...
1,1.569236E+00 1.686193E+00 -2.226649E-01 4.8578...
2,-3.606934E-01 1.794174E+00 2.647381E-01 -4.722...
3,-6.911299E-01 -1.063802E+00 -1.198263E+00 6.91...
4,-6.743614E-02 -6.367618E-01 -6.201658E-01 -6.2...
...,...
499995,-9.592633E-01 -4.784562E-01 -1.629655E+00 8.13...
499996,-6.288500E-01 -1.004971E+00 6.226089E-01 -4.14...
499997,9.331346E-01 6.243826E-01 1.168251E+00 1.25095...
499998,1.494034E+00 1.010618E+00 -5.078426E-01 1.0761...


In [64]:
"%signal: ", smalldataset.index.values.sum()/500_000

('%signal: ', 0.500572)